# Comparing European Mortality Rates to US States
Based on Nate Silver [wondering how US States compare to European countries](https://twitter.com/NateSilver538/status/1247259557476339717).

In [1]:
import pandas as pd
pd.options.display.max_rows = 100

In [2]:
# get cases and deaths by country
# Note that these are daily numbers, not cumulative sums
world = pd.read_csv('https://opendata.ecdc.europa.eu/covid19/casedistribution/csv')
world.head()

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2018
0,06/04/2020,6,4,2020,29,2,Afghanistan,AF,AFG,37172386.0
1,05/04/2020,5,4,2020,35,1,Afghanistan,AF,AFG,37172386.0
2,04/04/2020,4,4,2020,0,0,Afghanistan,AF,AFG,37172386.0
3,03/04/2020,3,4,2020,43,0,Afghanistan,AF,AFG,37172386.0
4,02/04/2020,2,4,2020,26,0,Afghanistan,AF,AFG,37172386.0


In [3]:
# Recode 'United_Kingdom' as 'United Kingdom' for later matching
world.loc[world.geoId == 'UK','countriesAndTerritories'] = 'United Kingdom'

In [4]:
# reorganize data and compute mortality rates per Million
world = world.groupby(['countriesAndTerritories'])\
    .agg({'cases': sum, 'deaths': sum, 'popData2018': max})
world.rename(
    columns={'countriesAndTerritories': 'country',
             'popData2018': 'population'},inplace=True)
world['deathsPerM'] = 1E6 * world.deaths / world.population
world.head()

,cases,deaths,population,deathsPerM
countriesAndTerritories,,,,
Afghanistan,299,7,37172386.0,0.188312
Albania,361,21,2866376.0,7.326324
Algeria,1320,152,42228429.0,3.599471
Andorra,501,18,77006.0,233.748020
Angola,14,2,30809762.0,0.064914


In [5]:
# Read in the state level cases/deaths data from the NYT
states = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv')
states.head()

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0


In [6]:
# read in the county populations file which happens to have state populations as well
counties = pd.read_csv('county_populations.csv')

In [7]:
states = states.merge(counties,left_on='state',right_on='County')
states = states.groupby(['state']).agg({'cases': max, 'deaths': max, 'Population': max})
states['deathsPerM'] = 1E6 * states.deaths / states.Population
states.rename(columns= {'Population': 'population'},inplace=True)
states.head()

,cases,deaths,population,deathsPerM
state,,,,
Alabama,1841,45,4887871,9.206462
Alaska,185,4,737438,5.424185
Arizona,2269,64,7171646,8.924032
Arkansas,853,16,3013825,5.308868
California,15076,349,39557045,8.822701


In [8]:
# read in the list of countries in Europe
# this will give us many irrelevant columns but we really just want the names
# Also note that the country populations in this file are not identical to those
# in the world file so we'll ignore these
europe = pd.read_csv('https://raw.githubusercontent.com/ajturner/acetate/master/places/Countries-Europe.csv')
europe.head()

,zoom,name,abbreviation,ISO alpha 2,ISO alpha 3,ISO numeric,land area km,population,latitude,longitude,continent
0,3,Ukraine,Ukr.,UA,UKR,804,603700.0,45415596,49.0,32.0,eu
1,3,France,Fr.,FR,FRA,250,547030.0,64768389,46.0,2.0,eu
2,3,Spain,Spain,ES,ESP,724,504782.0,46505963,40.0,-4.0,eu
3,3,Sweden,Swe.,SE,SWE,752,449964.0,9045000,62.0,15.0,eu
4,3,Germany,Ger.,DE,DEU,276,357021.0,82369000,51.5,10.5,eu


In [9]:
europe[europe.name=='United Kingdom']

,zoom,name,abbreviation,ISO alpha 2,ISO alpha 3,ISO numeric,land area km,population,latitude,longitude,continent
9,3,United Kingdom,U.K.,GB,GBR,826,244820.0,62348447,54.9,-3.12,eu


In [10]:
# merge the European data with the world countries, this will leave just the European data
europe = europe[['name','continent']]\
    .merge(world,left_on='name',right_index=True)\
    .drop(columns='continent')\
    .set_index('name')

In [11]:
# Merge the European data with the state data
europe_v_states = europe.append(states)

In [12]:
# sort by descending mortality
europe_v_states.sort_values(by='deathsPerM',ascending=False)

,cases,deaths,population,deathsPerM
Spain,130759,12418,46723749.0,265.774906
Italy,128948,15889,60431283.0,262.926736
Andorra,501,18,77006.0,233.748020
New York,122911,4161,19542209.0,212.923728
Belgium,19691,1447,11422068.0,126.684590
France,70478,8078,66987244.0,120.590123
New Jersey,37505,917,8908520.0,102.935168
Louisiana,13010,477,4659978.0,102.360998
Netherlands,16627,1651,17231017.0,95.815587
Switzerland,21065,715,8516543.0,83.954252
